In [1]:
#https://github.com/EmilStenstrom/conllu
#https://github.com/UniversalDependencies/UD_English-EWT
from conllu import parse, parse_tree
from collections import OrderedDict
import numpy as np

In [7]:
with open('../UD_Ukrainian-IU/uk_iu-ud-train.conllu') as f:
    c= f.read()
    trees= parse(c)

In [117]:
tree[5]

OrderedDict([('id', 6),
             ('form', 'була'),
             ('lemma', 'бути'),
             ('upostag', 'VERB'),
             ('xpostag', 'Vapis-sf'),
             ('feats',
              OrderedDict([('Aspect', 'Imp'),
                           ('Gender', 'Fem'),
                           ('Mood', 'Ind'),
                           ('Number', 'Sing'),
                           ('Tense', 'Past'),
                           ('VerbForm', 'Fin')])),
             ('head', 0),
             ('deprel', 'root'),
             ('deps', None),
             ('misc', OrderedDict([('Id', '0008')]))])

In [26]:
features=[]
for tree in trees:
    for node in tree:
        pos, lemma = get_pymorphy_features(node['form'])
        features.append({
            'word':node['form'],
            'golden_features':{
                'lemma':node['lemma'].lower(),
                'tag':str(node['upostag'])},
            'features':{
                'lemma':lemma,
                'tag':str(pos)}})

In [31]:
words_count= len(features)
words_count

75098

In [28]:
features[0]

{'features': {'lemma': 'у', 'tag': 'ADP'},
 'golden_features': {'lemma': 'у', 'tag': 'ADP'},
 'word': 'У'}

In [29]:
lemma_error=[]
pos_error=[]
for feature in features:
    if feature['golden_features']['lemma']!=feature['features']['lemma']:
        lemma_error.append((feature['golden_features']['lemma'],feature['features']['lemma']))
    if feature['golden_features']['tag']!=feature['features']['tag']:
        pos_error.append((feature['golden_features']['tag'],feature['features']['tag']))

In [38]:
from collections import Counter
def get_results(errors,words_count):
    print('correct:',(words_count-len(errors))/words_count)
    cnt = Counter(errors)
    return cnt.most_common(10)

In [39]:
get_results(pos_error,words_count)

correct: 0.6115742096993262


[(('PUNCT', 'None'), 14052),
 (('DET', 'PRON'), 2049),
 (('PROPN', 'NOUN'), 1959),
 (('ADP', 'INTJ'), 1178),
 (('ADP', 'NOUN'), 948),
 (('ADV', 'PART'), 662),
 (('PRON', 'DET'), 661),
 (('SCONJ', 'PART'), 589),
 (('ADP', 'ADV'), 528),
 (('NUM', 'None'), 504)]

In [40]:
get_results(lemma_error,words_count)

correct: 0.9310767264108232


[(('я', 'мен'), 139),
 (('його', 'йога'), 124),
 (('бути', 'булий'), 104),
 (('він', 'йога'), 102),
 (('те', 'той'), 93),
 (('рік', 'рок'), 82),
 (('все', 'весь'), 81),
 (('людина', 'люди'), 78),
 (('вона', 'її'), 73),
 (('те', 'тога'), 59)]

In [30]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer(lang='uk')
DET = ['інакший', 'його', 'тамтой', 'чий', 'їх', 'інш.', 'деякий', 'ввесь', 'ваш', 
     'ніякий', 'весь', 'інший', 'чийсь', 'жадний', 'другий', 'кожний', 
     'такий', 'оцей', 'скілька', 'цей', 'жодний', 'все', 'кілька', 'увесь', 
     'кожній', 'те', 'сей', 'ін.', 'отакий', 'котрий', 'усякий', 'самий', 
     'наш', 'усілякий', 'будь-який', 'сам', 'свій', 'всілякий', 'всенький', 'її', 
     'всякий', 'отой', 'небагато', 'який', 'їхній', 'той', 'якийсь', 'ин.', 'котрийсь', 
     'твій', 'мій', 'це']
mapping = {"ADJF": "ADJ", "ADJS": "ADJ", "COMP": "ADJ", "PRTF": "ADJ",
           "PRTS": "ADJ", "GRND": "VERB", "NUMR": "NUM", "ADVB": "ADV",
           "NPRO": "PRON", "PNCT": "PUNCT", "PRED": "ADV", "PREP": "ADP",
           "PRCL": "PART"}

def normalize_pos(token):
    if token.word in DET:
        return 'DET'
    
    if token.tag.POS == "CONJ":
        if "coord" in token.tag:
            return "CCONJ"
        else:
            return "SCONJ"
    else:
        return mapping.get(token.tag.POS, token.tag.POS)

def get_pymorphy_features(word):
    word_info = morph.parse(word)[0]
    pos = normalize_pos(word_info)
    return pos, word_info.normal_form

get_pymorphy_features('добро')

('NOUN', 'добра')